In [2]:
import requests
from  bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
import progressbar

title = []
date = []
link = []
content = []

mydatetime = datetime.date(2016, 1, 1)
url_date = []
for k in range(0, 366):
    date2016 = mydatetime + datetime.timedelta(k)
    date2016 = str(date2016)
    date2016 = date2016.replace("-", "")
    url_date.append(date2016)


def scraping(turl, title, date, link):
    source_code = requests.get(turl)
    plain_text = source_code.content
    soup = BeautifulSoup(plain_text, 'lxml', from_encoding='utf-8')
    page = soup.find('ul', class_='paginate_num')
    page_length = len(page.find_all('li')) - 2

    for x in range(1, page_length+1):
        url = turl + "&pn=" + str(x)
        source_code = requests.get(url)
        plain_text = source_code.content
        soup = BeautifulSoup(plain_text, 'lxml', from_encoding='utf-8')
        body = soup.find('div', id='list_area')
        article = body.find_all('dl', class_='list_item')

        for i in article:
            title.append(i.find('dt').find('a', href=True).get_text())
            date.append(i.find('span', class_='date').get_text())
            link.append(i.find('dt').find('a', href=True)['href'])

In [50]:
with progressbar.ProgressBar(max_value=len(url_date)) as bar:
    for x in url_date:
        url = "http://news.chosun.com/svc/list_in/list.html?catid=2&indate=" + str(x)
        scraping(url, title, date, link)
        # print(url_date.index(x) , "/" ,len(url_date) )
        time.sleep(2)
        bar.update(url_date.index(x))

chosun = pd.DataFrame({'title': title, 'date': date, 'link': link})

100% (366 of 366) |###############################################################| Elapsed Time: 0:20:58 Time: 0:20:58


In [9]:
turl= "http://news.chosun.com/site/data/html_dir/2016/09/08/2016090801790.html"
source_code = requests.get(turl)
plain_text = source_code.content
soup =BeautifulSoup(plain_text, 'lxml', from_encoding = 'utf-8')
article = soup.find('div', class_ = 'par')
article.get_text()

' 서울시 청년수당 대상자 가운데 연봉 2억원 이상 고소득 가정 출신이 있는 것으로 드러나 비판이 일고 있다. 서울시의회 이숙자(새누리 서초2) 의원은 동작구에 사는 A(27)씨 부양자는 1월부터 6월까지 월 평균 건강보험료(직장가입자)로 53만 9160원을 냈다고 8일 밝혔다. A씨는 38개월째 미취업 상태로, 구직 관련 자격증 학원에 등록하고 단기알바를 하겠다는 계획서를 냈다. 강북구 B(25)씨는 직장가입자인 부양자가 낸 월 평균 건강보험료가 53만 2440원이다. 미취업기간은 41개월이며, 취업과 자기계발이 목표다. 국민건강보험공단은 “건강보험료로 역산하면 연 소득이 2억원이 넘는 수준이다”라고 밝혔다. 성북구 C(26)씨는 지역가입자인 부양자의 건강보험료가 170만원, 송파구 D(27)씨는 116만원이다. 이들의 미취업기간은 48개월, 47개월이다. 연 소득이 7000만원을 넘는 수준인 직장가입자 건강보험료 월 18만원 이상인 가정 출신의 대상자는 40명에 달했다. 이숙자 의원은 “복지부와 협조하지 않으면 이런 문제가 생길 수 있어 공정성을 담보할 수 없다고 지적했는데 서울시가 강행했다”며 “급하게 추진하는 과정에 대상자 선정 부분을 소홀히 한 것”이라고 했다. 서울시는 전체 청년수당 대상자들의 건강보험료는 약 월 5만 2000원으로, 3인가구 기준 월 소득 167만원, 중위 소득 50% 수준이라고 밝혔다. 대상자 선정시 저소득층과 장기미취업자를 우선으로 하며 각각 50% 점수를 배정하다 보니 미취업기간이 긴 경우에는 소득이 높아도 뽑힌 경우가 있다고 서울시는 설명했다. 서울시는 또 건강보험료 상위 100명은 미취업 기간이 53개월에 달한다고 말했다. 전체 평균은 약 28개월이다. 서울시 관계자는 “내년에 사업을 하게 되면 이런 부분을 보완해 오해 소지가 없도록 하겠다”고 했다.'

In [3]:
chosun = pd.read_table("chosun.txt", encoding = "utf-8")
chosun["content"] = ""

In [18]:
def scrap_contents(data, start, finish):
    with progressbar.ProgressBar(max_value=finish-start) as bar:
        for i, x in enumerate(data.link[start:finish]):
            url = x
            source_code = requests.get(url)
            plain_text = source_code.content
            soup = BeautifulSoup(plain_text, 'lxml', from_encoding='utf-8')
            article = soup.find('div', class_ = 'par')
            data.content[i+start] += article.get_text()
            time.sleep(0.1)
            bar.update(i)

In [36]:
scrap_contents(chosun, 11472, len(chosun)-1)

#111469? 근처에서 오류

100% (560 of 560) |###############################################################| Elapsed Time: 0:01:34 Time: 0:01:34


In [37]:
chosun

,date,link,title,content
0,2016.01.01 (금),http://news.chosun.com/site/data/html_dir/2016...,"3당 전락 위기의 文, 뒤늦게 호남 구애","더불어민주당 문재인 대표가, 새해 첫날 고 김대중 전 대통령의 부인 이희호 여사를..."
1,2016.01.01 (금),http://news.chosun.com/site/data/html_dir/2016...,"반기문 ""박 대통령의 위안부 협상 타결, 역사가 높게 평가할 것""",반기문 UN 사무총장은 1일 한일 간 일본군 위안부 문제 협상과 관련해 “박근혜 대...
2,2016.01.01 (금),http://news.chosun.com/site/data/html_dir/2016...,"'누리 예산' 편성 놓고 몸싸움, 난장판 된 경기도의회",☞이 기사와 관련된 TV조선 영상 보기. 2015년의 마지막 날인 지난 31일 밤...
3,2016.01.01 (금),http://news.chosun.com/site/data/html_dir/2016...,"정부 ""남북 대화의 문 열어놓겠다""..北 신년사에 대한 입장 표명","정부는 1일 북한 김정은 노동당 제1비서의 신년사와 관련, “남북 간 대화의 문을 ..."
4,2016.01.01 (금),http://news.chosun.com/site/data/html_dir/2016...,野 단배식에서 호남 명물 홍어 사라져,1일 서울 여의도 더불어민주당사에서 열린 당 단배식(團拜式)에서 호남 명물인 홍어가...
5,2016.01.01 (금),http://news.chosun.com/site/data/html_dir/2016...,"[단독] ""북한 김경희 피살설, 권력층 내부에 퍼지고 있다""",북한 김정은 노동당 제1비서의 고모이자 처형된 장성택 노동당 행정부장의 부인인 김경...
6,2016.01.01 (금),http://news.chosun.com/site/data/html_dir/2016...,북한 김정은의 신년사 패션은,북한 김정은 노동당 제1비서의 2016년 신년사 패션은 ‘뿔테 안경에 인민복 차림’...
7,2016.01.01 (금),http://news.chosun.com/site/data/html_dir/2016...,北 김정은 노동당 제1비서 신년사 전문,다음은 김정은 북한 노동당 제1비서의 1일 신년사 전문. 친애하는 동지들! 우리는 ...
8,2016.01.01 (금),http://news.chosun.com/site/data/html_dir/2016...,"[속보] 北 김정은 ""남조선, 자주 통일에 앞장서라""…核 언급 없었다",
9,2016.01.01 (금),http://news.chosun.com/site/data/html_dir/2016...,"도쿄신문 ""김정일 '韓 심장부에 정보망 심으라' 스파이 교육 지시""",2011년 사망한 김정일 북한 국방위원장이 “적(한국)의 심장부에 정보 조직이 깊숙...


In [ ]:
http://news.chosun.com/site/data/html_dir/2016/12/16/2016121601620.html
http://news.chosun.com/site/data/html_dir/2016/12/16/2016121602854.html
url = x
source_code = requests.get(url)
plain_text = source_code.content
soup = BeautifulSoup(plain_text, 'lxml', from_encoding='utf-8')
            article = soup.find('div', class_ = 'par')
            data.content[i+start] += article.get_text()
            time.sleep(0.2)
            bar.update(i)

In [34]:
import numpy as np

np.where(chosun.content)

(array([    0,     1,     2, ..., 11469, 11470, 11471], dtype=int64),)

In [38]:
chosun.to_csv("chosun_final.csv")